## Installing Dependencies

The required libraries are installed to work with LangChain and connect to local models through Ollama. Without these dependencies, the rest of the notebook cannot run.

In [45]:
%pip install -U langchain langchain-community langchain_Ollama 


Note: you may need to restart the kernel to use updated packages.


## Basic Agent with a Custom Tool

A minimal agent is built to demonstrate the full flow: loading a model, registering a tool, and getting a response. This serves as a starting point before adding complexity to the system.

In [46]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_ollama import ChatOllama

@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = ChatOllama(
    model="qwen2.5:7b",      # debe coincidir con `ollama list`
    temperature=0.2,
)

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

out = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
print(out)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='a0178d29-b1e8-4034-890c-9bb99db970f4'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2026-02-22T04:33:11.1193794Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14126779300, 'load_duration': 174079400, 'prompt_eval_count': 141, 'prompt_eval_duration': 10583668800, 'eval_count': 20, 'eval_duration': 3344267400, 'logprobs': None, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--019c839f-3c9f-7cd2-aeac-9b79692b43c5-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'f64c1217-58d8-4381-aa8c-6b392568bb13', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 141, 'output_tokens': 20, 'total_tokens': 161}), ToolMessage(content="It's always sunny in sf!", name='get_weather', id='5158ec7d-d027-4e0e-95ed-a11984e876b2', tool_call_id='f64c1217-58d8-4381-aa8c-

## Defining the System Prompt

The agent's personality, role, and behavioral instructions are established here. The system prompt determines how the agent reasons and when it should use each available tool.

In [47]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

## Tools with User Context

The tools the agent can invoke are defined here, including a context schema that allows passing user information (such as their ID) securely at runtime, without exposing it directly in the prompt.

In [48]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

## Language Model Configuration

The local model (Qwen 2.5 via Ollama) is configured here. This is the model the agent will use to reason, decide which tools to call, and generate final responses.

In [49]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen2.5:7b",   # debe coincidir con `ollama list`
    temperature=0.5,
)

## Structured Response Schema

The output format that the agent must follow when responding is defined here. This ensures the response is predictable and typed, making it easier to use in applications that need to process the agent's data reliably.

In [50]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

## Conversation Memory

In-memory storage is enabled so the agent can remember the history of previous messages within a conversation thread, allowing multi-turn interactions with persistent context.

In [51]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## Assembling and Running the Full Agent

All previous components (model, tools, context, response format, and memory) are brought together into a single agent and tested with a two-turn conversation to verify that reasoning, memory, and structured output work correctly.

In [55]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather tomorrow?"}]},
    config=config,
    context=Context(user_id="1")
)

structured = response.get("structured_response")
print(structured if structured is not None else response["messages"][-1].content)
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)
structured = response.get("structured_response")
print(structured if structured is not None else response["messages"][-1].content)
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

Ha! Always sunny in Florida—except when it’s not. But don't worry, I've got you covered with a sunny side up prediction for tomorrow. Enjoy your day under the... well, mostly clear skies!

For more accurate weather conditions, feel free to ask again later or specify a different location if needed.
You're welcome! If you need any more punny weather updates, just holler. Stay sunny, even when it's not! 🌞😉
